In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter
from train import train
from train import train_rnis
from models_new import Renorm_Dynamic
from models_new import Rnis_Dynamic
from ei.EI_calculation import count_parameters
from dynamic_models_sis_new import Simple_Spring_Model
from datetime import datetime
t0 = datetime.now()


use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device

device(type='cuda', index=0)

# Generate data

In [3]:
mul_batch_size = [0,5000,3000,1000]#[0,10,4500,1500]
sigma = 0.03
rou = -0.5
steps = 7
dt = 0.01

seed = 2050
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
spring = Simple_Spring_Model(device=device)
test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=dt) 
train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=dt)

# Train RNIS

In [ ]:
sz = 4
scale = 2
L = 1
mae2_w = 3
T_total = 30001
hidden_units_dyn = 32
eis_rnis, term1s_rnis, term2s_rnis, losses_rnis, MAEs_mstep_rnis, net_rnis = train_rnis(train_data=train_data, test_data=test_data, 
                                                                    sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                    mae2_w=mae2_w, hidden_units_dyn = hidden_units_dyn, dt=dt, T2=T_total)

33312
check point-------o_0-> 16:34:22.219974; lasting 0:00:05.909153 seconds
Epoch: 0
Train loss: 0.4543
dEI: 0.8603
term1: 0.2799
term2: 1.4407
Test multistep loss: 0.5179
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 16:34:30.659683; lasting 0:00:08.439709 seconds
check point-------o_0-> 16:34:41.149832; lasting 0:00:10.490149 seconds
Epoch: 500
Train loss: 0.0550
dEI: 1.5095
term1: 1.5981
term2: 1.4208
Test multistep loss: 0.0531
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 16:34:51.550585; lasting 0:00:10.400753 seconds
check point-------o_0-> 16:35:00.989356; lasting 0:00:09.438771 seconds
Epoch: 1000
Train loss: 0.0354
dEI: 1.7812
term1: 2.1334
term2: 1.4290
Test multistep loss: 0.0335
---------------------------------------------------------------------------------------------------

# Train NIS

In [ ]:
sz = 4
scale = 2
L = 1
mae2_w = 3
T_total = 30001
eis_nis, term1s_nis, term2s_nis, losses_nis, MAEs_mstep_nis, net_nis = train(train_data=train_data, test_data=test_data, 
                                                                    sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                    mae2_w=mae2_w, dt=dt, T2=T_total,framework='nis')

# Train NIS+

In [ ]:
eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net_nisp = train(train_data=train_data, test_data=test_data, 
                                                                         sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                         mae2_w=mae2_w, dt=dt, T2=T_total,framework='nis+')

# Results

In [ ]:
print(count_parameters(net_rnis),count_parameters(net_nis),count_parameters(net_nisp))

In [ ]:
plt.figure(dpi=150)
timeseries = np.arange(len(eis_nis))*500
plt.plot(timeseries, eis_nis, label='NIS')
plt.plot(timeseries, eis_nisp, label='NIS+')
plt.plot(timeseries, eis_rnis, label='RNIS')
plt.title(r'$\mathcal{J}$')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(5,4),dpi=150)
timeseries = np.arange(len(eis_nis))*500
plt.plot(timeseries, losses_nis, label='NIS')
plt.plot(timeseries, losses_nisp, label='NIS+')
plt.plot(timeseries, losses_rnis, label='RNIS')
plt.title(r'loss')
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(5,4),dpi=150)
timeseries = np.arange(len(eis_nis))*500
plt.plot(timeseries, MAEs_mstep_nis, label='NIS')
plt.plot(timeseries, MAEs_mstep_nisp, label='NIS+')
plt.plot(timeseries, MAEs_mstep_rnis, label='RNIS')
plt.title(r'multistep mae')
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
for name, param in net_nisp.flow.named_parameters():
    print(name, param.data)

In [ ]:
for name, param in net_rnis.flow.named_parameters():
    print(name, param.data)